# Notebook 2: Word Embeddings - Words as Vectors

## Making Computers Understand Word Meanings

**The Problem:** Computers see words as just text: "cat" = ['c','a','t']

**The Solution:** Turn words into numbers that capture meaning!

---

### What We'll Learn:
1. Why words need to be vectors
2. Train Word2Vec on Shakespeare
3. Find similar words
4. Visualize word relationships in 3D!
5. Do word math: king - man + woman = ?

In [1]:
# Install required packages
!pip install plotly gensim scikit-learn -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 10.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; 

In [2]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from gensim.models import Word2Vec
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from shakespeare_utils import download_shakespeare_sonnets, tokenize
import warnings
warnings.filterwarnings('ignore')

ERROR:_plotly_utils.optional_imports:Error importing optional module pandas
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/_plotly_utils/optional_imports.py", line 28, in get_module
    return import_module(name)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/importlib/__init__.py", line 90, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1387, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1331, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 935, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 999, in exec_module
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "/usr/local/lib/python3.12/dist-packages/pandas/__init__.py", line 37, in

ImportError: Plotly express requires pandas to be installed.

## Part 1: Why Do We Need Word Vectors?

Let's see the problem with treating words as just text:

In [ ]:
# Problem: How similar are these words?
word1 = "king"
word2 = "queen"
word3 = "book"

print("As text strings:")
print(f"  '{word1}' and '{word2}' share {len(set(word1) & set(word2))} letters")
print(f"  '{word1}' and '{word3}' share {len(set(word1) & set(word3))} letters")
print("\n❌ This doesn't capture meaning at all!")

print("\n" + "="*50)
print("\nWhat we want:")
print("  'king' and 'queen' should be VERY similar (both royalty)")
print("  'king' and 'book' should be LESS similar (different concepts)")
print("\n✅ Word vectors can do this!")

## Part 2: One-Hot Encoding (The Old Way)

First attempt: Give each word a unique number

In [ ]:
# Simple vocabulary
vocab = ["king", "queen", "man", "woman", "book"]

# One-hot encoding
def one_hot_encode(word, vocab):
    vector = [0] * len(vocab)
    if word in vocab:
        vector[vocab.index(word)] = 1
    return vector

# Encode our words
print("One-Hot Encoding:")
print("="*30)
for word in vocab:
    vec = one_hot_encode(word, vocab)
    print(f"{word:6} = {vec}")

print("\n❌ Problems:")
print("  1. Vectors are HUGE (one dimension per word)")
print("  2. All words are equally different from each other")
print("  3. Can't capture any relationships")

## Part 3: Word2Vec - Learning Meaning from Context

**Key Idea:** Words that appear in similar contexts have similar meanings!

"You shall know a word by the company it keeps" - J.R. Firth (1957)

In [ ]:
# Load Shakespeare text
text = download_shakespeare_sonnets()
words = tokenize(text)

# Prepare sentences for Word2Vec
# Split into sentences (simple approach)
sentences = []
current_sentence = []

for word in words:
    current_sentence.append(word)
    # End sentence at punctuation or after 15 words
    if len(current_sentence) >= 15 or word in ['.', '!', '?']:
        if len(current_sentence) > 2:  # Keep sentences with at least 3 words
            sentences.append(current_sentence[:])
        current_sentence = []

print(f"Created {len(sentences)} sentences")
print(f"\nExample sentences:")
for i, sent in enumerate(sentences[:3]):
    print(f"  {i+1}. {' '.join(sent[:10])}...")

In [ ]:
# Train Word2Vec model
print("Training Word2Vec...")

# Better parameters for learning
model = Word2Vec(
    sentences=sentences,
    vector_size=50,      # Each word becomes a 50-number vector
    window=5,            # Look at 5 words before and after
    min_count=2,         # Lower threshold to include more words
    workers=1,           # Use 1 CPU core
    seed=42,             # For reproducibility
    epochs=100,          # More training iterations
    sg=1,                # Use skip-gram (better for small datasets)
    alpha=0.025,         # Learning rate
    min_alpha=0.0001     # Minimum learning rate
)

print(f"✅ Trained on {len(model.wv)} words")
print(f"Each word is now a {model.wv.vector_size}-dimensional vector!")

# Show an example vector
if 'love' in model.wv:
    love_vector = model.wv['love']
    print(f"\nThe word 'love' as a vector (first 10 numbers):")
    print(np.round(love_vector[:10], 3))

## Part 4: Finding Similar Words

Now we can find words with similar meanings!

In [ ]:
def find_similar_words(word, model, top_n=10):
    """Find words with similar meanings."""
    if word not in model.wv:
        print(f"Word '{word}' not in vocabulary!")
        return []

    similar = model.wv.most_similar(word, topn=top_n)
    return similar

# Test with different words - use more common words from sonnets
test_words = ['love', 'time', 'beauty', 'death']

for word in test_words:
    if word in model.wv:
        print(f"\nWords similar to '{word}':")
        similar = find_similar_words(word, model, top_n=5)
        for sim_word, score in similar:
            bar = '█' * int(score * 10)
            print(f"  {sim_word:12} {bar} {score:.2f}")
    else:
        print(f"\nWord '{word}' not in vocabulary")

In [ ]:
# Visualize similarity scores
def plot_similarity(word, model):
    similar = find_similar_words(word, model, top_n=10)

    if not similar:
        return

    words, scores = zip(*similar)

    fig = go.Figure(data=[
        go.Bar(
            x=list(scores),
            y=list(words),
            orientation='h',
            marker=dict(
                color=list(scores),
                colorscale='Viridis',
                showscale=False
            ),
            text=[f'{s:.2f}' for s in scores],
            textposition='auto',
        )
    ])

    fig.update_layout(
        title=f"Words Most Similar to '{word}'",
        xaxis_title="Similarity Score (0-1)",
        yaxis_title="Word",
        height=400,
        xaxis=dict(range=[0, 1])
    )

    fig.show()

# Visualize - use a common word from sonnets
plot_similarity('love', model)

## Part 5: Visualizing Word Space in 3D

Let's see how words cluster in space!

In [ ]:
# Get interesting words to visualize
categories = {
    'emotions': ['love', 'hate', 'fear', 'joy', 'sorrow'],
    'time': ['time', 'day', 'night', 'hour', 'year'],
    'nature': ['sun', 'moon', 'star', 'sky', 'earth'],
    'people': ['man', 'woman', 'youth', 'friend', 'lord']
}

# Collect words and their vectors
words_to_plot = []
vectors = []
colors = []
color_map = {'emotions': 'red', 'time': 'blue', 'nature': 'green', 'people': 'purple'}

for category, word_list in categories.items():
    for word in word_list:
        if word in model.wv:
            words_to_plot.append(word)
            vectors.append(model.wv[word])
            colors.append(color_map[category])

# Reduce to 3D using PCA
vectors_array = np.array(vectors)
pca = PCA(n_components=3, random_state=42)
vectors_3d = pca.fit_transform(vectors_array)

print(f"Visualizing {len(words_to_plot)} words in 3D space")

In [ ]:
# Create interactive 3D plot
fig = go.Figure(data=[go.Scatter3d(
    x=vectors_3d[:, 0],
    y=vectors_3d[:, 1],
    z=vectors_3d[:, 2],
    mode='markers+text',
    text=words_to_plot,
    textposition='top center',
    marker=dict(
        size=8,
        color=colors,
        opacity=0.8,
        line=dict(width=1, color='white')
    ),
    hovertemplate='%{text}<extra></extra>'
)])

fig.update_layout(
    title="Word Embeddings in 3D Space (Drag to Rotate!)",
    scene=dict(
        xaxis_title="Dimension 1",
        yaxis_title="Dimension 2",
        zaxis_title="Dimension 3",
        bgcolor='white'
    ),
    height=600,
    showlegend=False
)

# Add legend manually
for category, color in color_map.items():
    fig.add_trace(go.Scatter3d(
        x=[None], y=[None], z=[None],
        mode='markers',
        marker=dict(size=10, color=color),
        name=category.capitalize(),
        showlegend=True
    ))

fig.show()

print("🎯 Notice how similar words cluster together!")

## Part 6: Word Arithmetic - The Magic of Vectors!

With vectors, we can do math on words!

Famous example: **king - man + woman = queen**

In [ ]:
def word_arithmetic(positive_words, negative_words, model, top_n=5):
    """
    Perform word arithmetic.
    Result = sum(positive_words) - sum(negative_words)
    """
    # Check all words exist
    all_words = positive_words + negative_words
    for word in all_words:
        if word not in model.wv:
            print(f"Word '{word}' not in vocabulary!")
            return []

    # Do the arithmetic
    result = model.wv.most_similar(
        positive=positive_words,
        negative=negative_words,
        topn=top_n
    )

    return result

# Try some word math with words from sonnets!
print("Word Arithmetic Examples:")
print("="*40)

# Example 1 - use common words from sonnets
print("\n1. love - hate + beauty = ?")
if all(w in model.wv for w in ['love', 'hate', 'beauty']):
    result = word_arithmetic(['love', 'beauty'], ['hate'], model, top_n=3)
    for word, score in result:
        print(f"   → {word} ({score:.2f})")
else:
    print("   Some words not in vocabulary")

# Example 2
print("\n2. day - night + sun = ?")
if all(w in model.wv for w in ['day', 'night', 'sun']):
    result = word_arithmetic(['day', 'sun'], ['night'], model, top_n=3)
    for word, score in result:
        print(f"   → {word} ({score:.2f})")
else:
    print("   Some words not in vocabulary")

# Example 3 - using words more likely to be in sonnets
print("\n3. youth - old + beauty = ?")
if all(w in model.wv for w in ['youth', 'old', 'beauty']):
    result = word_arithmetic(['youth', 'beauty'], ['old'], model, top_n=3)
    for word, score in result:
        print(f"   → {word} ({score:.2f})")
else:
    print("   Some words not in vocabulary")

## Part 7: Visualizing Word Relationships

Let's see how word vectors capture relationships!

In [ ]:
# Create a word similarity matrix
selected_words = ['love', 'hate', 'time', 'beauty', 'death', 'life',
                  'youth', 'age', 'sun', 'moon', 'day', 'night']

# Filter to words that exist
words_in_model = [w for w in selected_words if w in model.wv]

# Build similarity matrix
n = len(words_in_model)
similarity_matrix = np.zeros((n, n))

for i, word1 in enumerate(words_in_model):
    for j, word2 in enumerate(words_in_model):
        if i == j:
            similarity_matrix[i, j] = 1.0
        else:
            similarity_matrix[i, j] = model.wv.similarity(word1, word2)

# Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=similarity_matrix,
    x=words_in_model,
    y=words_in_model,
    colorscale='RdBu',
    zmid=0,
    text=np.round(similarity_matrix, 2),
    texttemplate='%{text}',
    textfont={"size": 8},
    colorbar=dict(title="Similarity")
))

fig.update_layout(
    title="Word Similarity Matrix (Red = Similar, Blue = Different)",
    height=500,
    width=600,
    xaxis_title="Word",
    yaxis_title="Word"
)

fig.show()

print("🎯 Notice the patterns:")
print("  - Opposites (love/hate, day/night) have low similarity")
print("  - Related concepts (sun/day, moon/night) have high similarity")

## Part 8: Using Embeddings for Next Word Prediction

How do embeddings help with predicting the next word?

In [ ]:
def predict_next_word_with_embeddings(context_words, model, top_n=5):
    """
    Predict next word based on context similarity.
    """
    # Check if context words exist
    valid_words = [w for w in context_words if w in model.wv]

    if not valid_words:
        print("No valid context words found!")
        return []

    # Get average vector of context
    context_vector = np.mean([model.wv[w] for w in valid_words], axis=0)

    # Find most similar words to context
    similar_words = model.wv.similar_by_vector(context_vector, topn=top_n+len(valid_words))

    # Filter out the context words themselves
    predictions = [(w, s) for w, s in similar_words if w not in valid_words][:top_n]

    return predictions

# Test prediction
print("Predicting next word using embeddings:")
print("="*40)

contexts = [
    ['the', 'beautiful'],
    ['love', 'and'],
    ['in', 'the', 'dark']
]

for context in contexts:
    print(f"\nContext: {' '.join(context)} ___")
    predictions = predict_next_word_with_embeddings(context, model)
    print("Predictions:")
    for word, score in predictions:
        bar = '█' * int(score * 10)
        print(f"  {word:10} {bar} {score:.2f}")

## Part 9: Interactive Embedding Explorer

Explore word relationships interactively!

In [ ]:
def explore_word(word, model):
    """
    Interactive word exploration.
    """
    if word not in model.wv:
        print(f"❌ Word '{word}' not found in vocabulary!")
        print("Try common words like: love, time, beauty, death, life")
        return

    print(f"\n🔍 Exploring: '{word}'")
    print("="*50)

    # Vector info
    vector = model.wv[word]
    print(f"\n📊 Vector Statistics:")
    print(f"  Dimensions: {len(vector)}")
    print(f"  Mean: {np.mean(vector):.3f}")
    print(f"  Std: {np.std(vector):.3f}")
    print(f"  Range: [{np.min(vector):.3f}, {np.max(vector):.3f}]")

    # Similar words
    print(f"\n🤝 Most Similar Words:")
    similar = model.wv.most_similar(word, topn=5)
    for sim_word, score in similar:
        bar = '█' * int(score * 10)
        print(f"  {sim_word:12} {bar} {score:.2f}")

    # Opposite words (least similar)
    print(f"\n🔄 Most Different Words:")
    all_words = list(model.wv.index_to_key)[:100]  # Check top 100 words
    similarities = [(w, model.wv.similarity(word, w)) for w in all_words if w != word]
    similarities.sort(key=lambda x: x[1])

    for opp_word, score in similarities[:5]:
        bar = '█' * max(0, int((1 + score) * 5))  # Adjust for negative scores
        print(f"  {opp_word:12} {bar} {score:.2f}")

# Try it!
explore_word('love', model)
print("\n" + "="*50)
explore_word('death', model)

## Summary: What We Learned About Word Embeddings

### Key Concepts:
1. **Words as Vectors**: Each word becomes a list of numbers
2. **Similarity**: Similar words have similar vectors
3. **Context Matters**: Words learn meaning from their neighbors
4. **Word Math**: We can add and subtract word meanings!

### Why Embeddings are Better than Counting:

| Aspect | N-grams (Counting) | Word Embeddings |
|--------|-------------------|------------------|
| **Similarity** | ❌ Can't measure | ✅ Built-in |
| **Size** | Huge sparse matrices | Small dense vectors |
| **Generalization** | Only exact matches | Finds similar words |
| **Relationships** | None | Captures analogies |

### Applications:
- 🔍 Search engines (find related terms)
- 💬 Chatbots (understand user intent)
- 🌍 Translation (match concepts across languages)
- 📝 Text generation (better predictions)

### Next Steps:
- **Notebook 3**: Use embeddings in a neural network
- **Notebook 4**: Compare all methods for next-word prediction

## Bonus: Create Your Own Word Analogy!

Try creating your own word equations!

In [ ]:
def interactive_word_math():
    """
    Interactive word arithmetic playground.
    """
    print("🧮 Word Math Playground")
    print("="*40)
    print("Formula: result = positive_words - negative_words")
    print("Example: king - man + woman = queen\n")

    # Some examples to try
    examples = [
        (["summer", "hot"], ["winter"], "What's hot in summer but not in winter?"),
        (["paris", "france"], ["london"], "Paris is to France as London is to?"),
        (["good", "best"], ["bad"], "Good is to best as bad is to?")
    ]

    for pos, neg, hint in examples:
        print(f"\n{hint}")
        print(f"Calculating: {' + '.join(pos)} - {' - '.join(neg)}")

        # Check if words exist
        all_words = pos + neg
        valid = all([w in model.wv for w in all_words])

        if valid:
            result = word_arithmetic(pos, neg, model, top_n=3)
            print("Results:")
            for word, score in result:
                print(f"  → {word} ({score:.2f})")
        else:
            missing = [w for w in all_words if w not in model.wv]
            print(f"  ❌ Missing words: {missing}")

interactive_word_math()